
# SchObel -Zhull-Hull-White model 

## 1. Overview of the Models

Your code combines several elements:
- **Stochastic Volatility:** A Heston‐like model (here in the spirit of Schobel and Zhu) for the stock’s volatility.
- **Stochastic Interest Rates:** A Hull–White model that describes the evolution of the short rate.
- **SZHW Model:** This hybrid model (sometimes called the SZHW model) combines stochastic volatility with stochastic interest rates. The goal is to capture both the volatility smile and the term structure of interest rates.

The pricing of European options is performed using the **COS method**, which relies on the characteristic function (CF) of the logarithm of the asset price.

---

## 2. The COS Method and Option Pricing

### 2.1. The COS Method

The COS method expresses the price of an option as a Fourier-cosine series expansion. If the characteristic function (CF) of the log-price is \$\varphi(u)\$, then the option price can be computed by  
$$
V(K) = e^{-rT} \sum_{k=0}^{N-1} \Re\left\{\varphi\left(u_k\right)e^{-iu_k a}\right\} H_k,
$$  
with
- \$u_k = \frac{k\pi}{b-a}\$
- \$[a,b]\$ is a truncation domain for the integration (typically chosen based on \$\sqrt{T}\$),
- \$H_k\$ are the cosine coefficients of the payoff function.

### 2.2. Function: \`CallPutOptionPriceCOSMthd_StochIR\`

This function implements the COS method. Key points:
- **Input parameters:**
  - **cf:** The characteristic function (denoted \$\varphi(u)\$).
  - **CP:** Option type (CALL or PUT).
  - **S0:** Initial stock price.
  - **tau:** Time to maturity.
  - **K:** List of strike prices.
  - **N:** Number of expansion terms.
  - **L:** Determines the truncation domain size.
  - **P0T:** The zero-coupon bond price for maturity T.

- **Steps and Formulas:**
  - **Log-moneyness:**
    $$
    x_0 = \ln\left(\frac{S_0}{K}\right)
    $$
  - **Truncation domain:**
    $$
    a = -L\sqrt{\tau},\quad b = L\sqrt{\tau}
    $$
  - **COS expansion frequencies:**
    $$
    u_k = \frac{k\pi}{b-a},\quad k=0,1,\ldots,N-1
    $$
  - **Coefficient Calculation:**  
    The function calls `CallPutCoefficients` (see below) to compute \$H_k\$, which are the cosine series coefficients for the payoff function.
  - **Option Value:**
    The value is computed as  
    $$
    V = K\, \Re\left\{ \exp\left(i (x_0 - a)u\right) \cdot \left[\varphi(u) H_k\right] \right\},
    $$
    with a half-weight given to the first term (\$k=0\$). Then, for calls, put-call parity is used.

---

## 3. Coefficient Functions for the COS Method

### 3.1. Function: \`CallPutCoefficients\`

Depending on the option type:
- **For Calls:**
  - Uses the interval \([c,d]=[0,b]\).
- **For Puts:**
  - Uses the interval \([c,d]=[a,0]\).

It then calls `Chi_Psi` to obtain the intermediate coefficients \$\chi_k\$ and \$\psi_k\$, and combines them as  
$$
H_k = \frac{2}{b-a}\left(\chi_k - \psi_k\right) \quad \text{(for calls)},
$$  
or  
$$
H_k = \frac{2}{b-a}\left(-\chi_k + \psi_k\right) \quad \text{(for puts)}.
$$

### 3.2. Function: \`Chi_Psi\`

This function calculates the integrals needed for the COS coefficients. In brief:
- **\(\psi_k\):** Derived from sine integrals representing the integral of cosine functions over the interval.
  
  For \$k\ge 1\$:  
  $$
  \psi_k = \frac{(b-a)}{k\pi}\left[\sin\left(\frac{k\pi (d-a)}{b-a}\right)-\sin\left(\frac{k\pi (c-a)}{b-a}\right)\right],
  $$
  
  and for \$k=0\$:
  $$
  \psi_0 = d-c.
  $$

- **\(\chi_k\):** Involves cosine integrals and exponential terms (accounting for the payoff function’s structure in the Black–Scholes framework).

The detailed expressions contain cosine and sine terms along with exponential factors. The function returns a dictionary with keys `"chi"` and `"psi"`.

---

## 4. Black–Scholes and Implied Volatility

### 4.1. Function: \`BS_Call_Put_Option_Price\`

This computes the Black–Scholes price for calls and puts using the well-known formulas:
- Define:
  $$
  d_1 = \frac{\ln(S_0/K) + \left(r+\frac{1}{2}\sigma^2\right)\tau}{\sigma\sqrt{\tau}},\quad d_2 = d_1 - \sigma\sqrt{\tau}.
  $$
- **Call Price:**
  $$
  C = S_0\,N(d_1) - Ke^{-r\tau}\,N(d_2),
  $$
- **Put Price:**
  $$
  P = Ke^{-r\tau}\,N(-d_2) - S_0\,N(-d_1),
  $$
where \$N(\cdot)\$ denotes the cumulative normal distribution.

### 4.2. Function: \`ImpliedVolatilityBlack76\`

This function finds the implied volatility (by inverting the Black–Scholes formula):
- It first creates a grid of volatility values.
- Then, it interpolates the option price to estimate an initial volatility value.
- Finally, it refines this estimate using a root-finding (Newton–Raphson) algorithm so that
  $$
  \text{BS\_Price}(\sigma) - \text{Market Price} = 0.
  $$

---

## 5. Characteristic Functions for the SZHW and BSHW Models

The next several functions build the characteristic functions (CF) for hybrid models.

### 5.1. Function: \`C(u,tau,lambd)\`

This function represents the part of the CF due to the short rate process in the Hull–White model:
$$
C(u,\tau) = \frac{(i\,u-1)}{\lambda}\left(1-e^{-\lambda\tau}\right).
$$

### 5.2. Function: \`D(u,tau,kappa,Rxsigma,gamma)\`

For the volatility process part, the function computes:
- First, define auxiliary terms:
  $$
  a_0 = -\frac{1}{2}u\,(i+u),\quad a_1 = 2\left(\gamma\, R_{xs}\, i\, u - \kappa\right),\quad a_2 = 2\,\gamma^2,
  $$
- Then, the discriminant:
  $$
  d = \sqrt{a_1^2 - 4a_0a_2},
  $$
- With
  $$
  g = \frac{-a_1-d}{-a_1+d},
  $$
- And finally:
  $$
  D(u,\tau) = \frac{-a_1-d}{2a_2\left(1-g\,e^{-d\tau}\right)}\Bigl[1-e^{-d\tau}\Bigr].
  $$

### 5.3. Function: \`E(u,tau,lambd,gamma,Rxsigma,Rrsigma,Rxr,eta,kappa,sigmabar)\`

This function further captures interactions and correction terms arising from:
- The correlation between the stochastic volatility process and the interest rate process.
- Integrated terms that involve additional exponential and logarithmic adjustments.
  
The returned value is used later in the exponent of the CF.

### 5.4. Function: \`A(u,tau,...)\`

This function combines several terms together into an integrated term:
- It defines an auxiliary constant \$c_1\$ and computes a series of correction functions (\(f_1,\dots,f_6\)).
- An initial term \$A_1\$ is computed:
  $$
  A_1 = \frac{1}{4}\left[(-a_1-d)\tau -2\ln\left(\frac{1-g\,e^{-d\tau}}{1-g}\right)\right] + f_6,
  $$
- Then, numerical integration (using trapezoidal rule via `integrate.trapz`) is performed over a grid \$z\in[0,\tau]\$, integrating the function
  $$
  f(z) = \left(\kappa\bar{\sigma}+\frac{1}{2}\gamma^2 E(u,z) + \gamma\eta\,R_{rs}\, C(u,z)\right)E(u,z),
  $$
- The final \$A(u)\$ is the sum of the integrated value and \$A_1\$.  
This term accounts for the effect of the time-dependent drift and variance corrections.

### 5.5. Function: \`ChFSZHW(u,P0T,sigma0,tau,...)\`

This function assembles the SZHW model’s characteristic function:
- It computes
  - \$v_D = D(u,\tau,\,\cdot)\$,
  - \$v_E = E(u,\tau,\,\cdot)\$,
  - \$v_A = A(u,\tau,\,\cdot)\$,
- Then, the initial variance \$v_0 = \sigma_0^2\$ is used.
- A correction term, based on the zero-coupon bond price, is applied:
  $$
  \text{correction} = (i\,u-1)\left(\ln\frac{1}{P_{0T}(\tau)} + \text{hlp}\right),
  $$
  with 
  $$
  \text{hlp} = \frac{\eta^2}{2\lambda^2}\left[\tau +\frac{2}{\lambda}\left(e^{-\lambda\tau}-1\right)-\frac{1}{2\lambda}\left(e^{-2\lambda\tau}-1\right)\right].
  $$
- Finally, the CF is given by:
  $$
  \phi(u) = \exp\Bigl(v_0\,v_D + \sigma_0\,v_E + v_A + \text{correction}\Bigr).
  $$

### 5.6. Function: \`ChFBSHW(u, T, P0T, lambd, eta, rho, sigma)\`

This is an alternative CF for the BSHW (Black–Scholes–Hull–White) model:
- It computes the short rate initial value \(r_0\) via numerical differentiation of the zero-coupon bond,
- Evaluates a function \(\theta(t)\) that fits the initial yield curve,
- Computes an integrated term based on the function
  $$
  C(u,T) = \frac{1}{\lambda}(i\,u-1)(1-e^{-\lambda T}),
  $$
- And finally outputs
  $$
  \phi(u) = \exp\left( A(u) + C(u,T)r_0 \right),
  $$
  where \(A(u)\) itself is assembled from several terms that involve numerical integration over the yield curve dynamics.

---

## 6. Main Calculation and Graphical Output

### 6.1. Function: \`mainCalculation\`

This function orchestrates the model evaluation and pricing, performing the following:
1. **Model Parameter Settings:**  
   Sets parameters for the Hull–White part (\(\lambda,\,\eta\)), the stock price (\(S_0,\,T\)), and for the SZHW model (including \(\sigma_0,\,\gamma,\,\kappa\), various correlations such as \(R_{xsigma}\) and \(R_{rsigma}\), and \(\bar{\sigma}\)).

2. **Strike and Zero-Coupon Curve Setup:**  
   - A grid of strike prices \(K\) is defined.
   - The zero-coupon bond price function is given by  
     $$
     P_{0T}(T) = e^{-0.025 T}.
     $$

3. **COS Method and Implied Volatility:**  
   - The forward stock price is calculated as  
     $$
     \text{forward} = \frac{S_0}{P_{0T}(T)}.
     $$
   - For several parameter variations (effects of \(\gamma\), \(\kappa\), \(R_{xsigma}\), and \(\bar{\sigma}\)), the function:
     - Evaluates the SZHW CF using \`ChFSZHW\`.
     - Prices options via the COS method using \`CallPutOptionPriceCOSMthd_StochIR\`.
     - Computes implied volatilities by inverting the Black–Scholes formula.
   - These implied volatilities are then plotted as a function of strike.

*Note:* The plots illustrate the sensitivity of the implied volatility smile/surface to variations in model parameters.

---

## 7. Summary

- **COS Method:**  
  Uses Fourier-cosine series expansion to efficiently price options given the characteristic function.

- **SZHW Model:**  
  Combines a stochastic volatility model (similar to Heston/Schobel–Zhu) with a Hull–White stochastic interest rate model. The CF for this model is computed by combining several integral components — represented by the functions \(C(u)\), \(D(u)\), \(E(u)\), and \(A(u)\) — together with a correction for the initial yield curve.

- **Black–Scholes & Implied Volatility:**  
  For benchmarking and calibration, Black–Scholes prices are computed, and implied volatility is extracted by inverting the Black–Scholes formula.

- **Numerical Integration:**  
  In many parts (e.g., in computing \(A(u)\) and the integration of \(\theta(t)\)), the trapezoidal rule is used via `integrate.trapz`.

